# Trabalho de Machine Learning

**Dados do Aluno**


* **Nome**: Denilson Santos
* **Disciplina**: Machine Learning
* **Pós-Graduação**: Ciência de dados e Analytics
----


# Dados Utilizados

O dataset utilizado foi os dados da ANAC -
Possui os seguintes campos:

* **Empresa Aérea**: Empresa Aérea responsável por operar as etapas;
* **Aeroporto de Origem**: Origem do voo
* **Aeroporto de Destino**: Destino do Voo
* **UF**
* **Mes e Ano**: Refere-se à data prevista para início da primeira etapa de cada voo.
* **Grupo**: Faz referência ao tipo de operação das etapas:
  1. **Improdutivas** (Non-revenue flights): etapas que não geraram receita à empresa aérea (como realização de treinamentos, voo para manutenção de aeronaves);
  2. **Regulares** (Scheduled revenue flights): etapas remuneradas que são realizadas sob uma numeração de Horário de Transporte (HOTRAN). Recebem esse nome, pois possuem a característica de serem realizadas regularmente;
  3. **Não Regulares** (Non-scheduled revenue flights): etapas remuneradas que não são realizadas sob uma numeração de Horário de Transporte (HOTRAN). Recebem esse nome, pois possuem a característica de serem realizadas de forma não continuada. Aqui estão os voos Charters, Fretamentos.
* **Assentos**: é o número de assentos disponíveis em cada etapa de voo de acordo com a configuração da aeronave na execução da etapa;
* **Passageiros Pagos**: Refere-se aos passageiros que ocupam assentos oferecidos e que geram receita, com a compra de assentos, para a empresa de transporte aéreo.

# Origem dos dados

**Link**:

https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/historico-de-voos

**Descrição**:

https://www.anac.gov.br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/dados-estatisticos-do-transporte-aereo/48-dados-estatisticos-do-transporte-aereo

# 1. Definição do Problema
* O problema é identificar qual companhia aérea corre o risco de falir.
* A hipótese é que as companhias aéreas com menos passageiros pagos e mais assentos disponíveis estão em risco.
* O dataset contém informações sobre várias companhias aéreas, incluindo o número de passageiros pagos e assentos disponíveis.


Para resolver esse problema, iremos utilizar um **modelo de regressão**, para encontrar qual companhia aérea corre o risco de falir.

#2. Preparação de Dados

Importando as Bibliotecas Python

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


from tabulate import tabulate
from csv import DictReader

from google.colab import data_table
data_table.enable_dataframe_formatter()

Configurando o Google Colab para exibir o resultado formatado

In [55]:
!mkdir -p ~/.ipython/profile_default
!echo "c = get_config()" > ~/.ipython/profile_default/ipython_config.py

Importação dos Dados

In [56]:
url = "https://raw.githubusercontent.com/Dssvsc/MVP_Machine_Learning/main/dados_anac.csv"
data = pd.read_csv(url, sep=';', on_bad_lines='warn')

data

,empresa,mes,ano,aeroporto_origem,uf_origem,aeroporto_destino,uf_destino,grupo,passageiros_pagos,assentos
0,AZU - Azul Linhas Aéreas Brasileiras,Janeiro,2016,SBAE - Arealva,SP,SBAE - Várzea Grande,MT,Não Regular,56.0,70.0
1,AZU - Azul Linhas Aéreas Brasileiras,Janeiro,2016,SBAE - Arealva,SP,SBAE - Campinas,SP,Regular,1938.0,2722.0
2,AZU - Azul Linhas Aéreas Brasileiras,Janeiro,2016,SBAE - Arealva,SP,SBAE - Marília,SP,Regular,1.0,1602.0
3,AZU - Azul Linhas Aéreas Brasileiras,Janeiro,2016,SBAE - Arealva,SP,SBAE - Ribeirão Preto,SP,Regular,57.0,70.0
4,AZU - Azul Linhas Aéreas Brasileiras,Janeiro,2016,SBAR - Aracaju,SE,SBAR - Confins,MG,Não Regular,1597.0,1998.0
...,...,...,...,...,...,...,...,...,...,...
145588,TAM - LATAM Linhas Aéreas,Dezembro,2022,SBVT - Vitória,ES,SBVT - Rio De Janeiro,RJ,Regular,9670.0,17330.0
145589,TAM - LATAM Linhas Aéreas,Dezembro,2022,SBVT - Vitória,ES,SBVT - São Paulo,SP,Improdutivo,0.0,0.0
145590,TAM - LATAM Linhas Aéreas,Dezembro,2022,SBVT - Vitória,ES,SBVT - São Paulo,SP,Não Regular,909.0,1548.0
145591,TAM - LATAM Linhas Aéreas,Dezembro,2022,SBVT - Vitória,ES,SBVT - São Paulo,SP,Regular,16602.0,30218.0


Tratamento dos dados

In [57]:
cols = ["passageiros_pagos","assentos"]
data[cols] = data[cols].replace("None", "0")

In [58]:
data['passageiros_pagos'] = data['passageiros_pagos'].astype(float)
data['assentos'] = data['assentos'].astype(float)

In [59]:
## Filtrando as colunas necessárias
data = data[['empresa', 'passageiros_pagos', 'assentos']]

In [60]:
imputer = SimpleImputer(strategy='mean')
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].fillna(data[col].mode()[0])
    else:
        data[col] = data[col].fillna(data[col].mean())

<ipython-input-60-8b38ac1c23ea>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].fillna(data[col].mode()[0])
<ipython-input-60-8b38ac1c23ea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].fillna(data[col].mean())


Separação do dataset entre treino e teste

In [61]:
# Codificando a coluna 'empresa'
le = LabelEncoder()
data['empresa'] = le.fit_transform(data['empresa'])

<ipython-input-61-efdb7f48bca1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['empresa'] = le.fit_transform(data['empresa'])


In [62]:
# Codificando variáveis categóricas
label_encoders = {col: LabelEncoder() for col in data.columns if data[col].dtype == 'object'}
for col, le in label_encoders.items():
    data[col] = le.fit_transform(data[col])

In [63]:
X = data.drop('empresa', axis=1)
y = data['empresa']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Normalizando os dados

In [64]:
from sklearn.preprocessing import LabelEncoder

# Cria um LabelEncoder para cada coluna categórica
label_encoders = {col: LabelEncoder() for col in X_train.columns if X_train[col].dtype == 'object'}


In [65]:
# Aplica o LabelEncoder em cada coluna categórica
for col, le in label_encoders.items():
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

In [66]:
# Agora você pode aplicar o StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. Modelagem e treinamento
Criando o modelo de regressão linear

In [67]:
model = LinearRegression()

Treinamento do modelo de dados

In [68]:
from sklearn.preprocessing import LabelEncoder

# Cria um LabelEncoder para cada coluna categórica
label_encoders = {col: LabelEncoder() for col in data.columns if data[col].dtype == 'object'}

# Aplica o LabelEncoder em cada coluna categórica
for col, le in label_encoders.items():
    data[col] = le.fit_transform(data[col])

# Agora você pode aplicar o SimpleImputer
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data)


In [69]:
model.fit(X_train, y_train)

LinearRegression()

# 4. Avaliação de Resultados
Avaliando o modelo com os dados de teste


In [70]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

MSE: 2.3561602044460552


Utilizando cross-validation

In [71]:
scores = cross_val_score(model, X, y, cv=5)
print(f'Cross-validation scores: {scores}')

Cross-validation scores: [ 0.          0.         -4.12012362 -0.69445622  0.        ]


Criação do pipeline

In [72]:
pipeline = Pipeline(steps=[('s',scaler),('m',model)])
pipeline.fit(X_train, y_train)
y_pred_pipeline = pipeline.predict(X_test)
mse_pipeline = mean_squared_error(y_test, y_pred_pipeline)

Resultados

In [73]:
print(f'MSE (com pipeline): {mse_pipeline}')

MSE (com pipeline): 2.3561602044460552


# 5. Resultado Final
Identificando a empresa com maior risco de falir

In [74]:
risk_index = y_pred.argmin()
risk_company = y[risk_index]
print(f'A empresa com maior risco de falir é: {risk_company}')

A empresa com maior risco de falir é: 0


In [75]:
# Identificando a empresa com maior risco de falir
risk_index = y_pred.argmin()
risk_company_encoded = y[risk_index]

In [76]:
# Use o LabelEncoder para obter o nome da empresa
risk_company = le.inverse_transform([risk_company_encoded])

print(f'A empresa com maior risco de falir é: {risk_company[0]}')

A empresa com maior risco de falir é: AZU - Azul Linhas Aéreas Brasileiras


# 6. Conclusão

O resultado final é que a `Azul Linhas Aéreas Brasileiras` corre o risco de falir segundo o treinamento do modelo de dados.

O meu maior desafio foi aprender a linguagem python e utilizar os recursos do Google Colab, além do GitHub.

Tive que fazer diversas pesquisas para entender e aprender a dominar todos esses recursos que eu não domino, principalmente em fazer um código limpo e descritivo.